# Pandas time-series
Feb 15th 2022

In [1]:
import sys
print(sys.executable)
import numpy as np
import pandas as pd
import os
print(os.getcwd())
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
# Config:
pd.options.display.float_format = '{:,.2f}'.format
plotsize = (13, 5)

/home/razielar/miniconda3/envs/time-series/bin/python
/home/razielar/Documents/git_rep/Machine_learning/time_series_forecasting/coursera_time_series/week_1


## Read data
4 years of daily sales

In [2]:
df= pd.read_excel('data/sample-superstore.xls')
print(df.shape)
df.head()

(9994, 21)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
1,2,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.58
2,3,CA-2013-138688,2013-06-13,2013-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.00,6.87
3,4,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58,5,0.45,-383.03
4,5,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37,2,0.20,2.52


In [3]:
# Pring column names:
for i,j in enumerate(df.columns):
    print(i+1, j)

1 Row ID
2 Order ID
3 Order Date
4 Ship Date
5 Ship Mode
6 Customer ID
7 Customer Name
8 Segment
9 Country
10 City
11 State
12 Postal Code
13 Region
14 Product ID
15 Category
16 Sub-Category
17 Product Name
18 Sales
19 Quantity
20 Discount
21 Profit


### Simplify Time-series data
Total Sales by order date and category

In [4]:
variables= ['Order Date', 'Category', 'Sales']
df.loc[:, variables]

,Order Date,Category,Sales
0,2013-11-09,Furniture,261.96
1,2013-11-09,Furniture,731.94
2,2013-06-13,Office Supplies,14.62
3,2012-10-11,Furniture,957.58
4,2012-10-11,Office Supplies,22.37
...,...,...,...
9989,2011-01-22,Furniture,25.25
9990,2014-02-27,Furniture,91.96
9991,2014-02-27,Technology,258.58
9992,2014-02-27,Office Supplies,29.60


In [5]:
# Group_by date and category and important feat sales:
base= df.groupby(['Order Date', 'Category'], as_index= False)['Sales'].sum()
base

,Order Date,Category,Sales
0,2011-01-04,Office Supplies,16.45
1,2011-01-05,Office Supplies,288.06
2,2011-01-06,Office Supplies,19.54
3,2011-01-07,Furniture,"2,573.82"
4,2011-01-07,Office Supplies,685.34
...,...,...,...
2859,2014-12-30,Office Supplies,282.44
2860,2014-12-30,Technology,302.38
2861,2014-12-31,Furniture,323.14
2862,2014-12-31,Office Supplies,299.72


In [6]:
base.dtypes

Order Date    datetime64[ns]
Category              object
Sales                float64
dtype: object

Let's check the number of years we have in this dataset

In [7]:
# Unique years we have in the dataset:
np.unique(np.array(base['Order Date'], dtype= 'datetime64[Y]'))

array(['2011', '2012', '2013', '2014'], dtype='datetime64[Y]')

In [9]:
# Monthly
tmp= np.unique(np.array(base['Order Date'], dtype= 'datetime64[M]'))
tmp

48


array(['2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12'],
      dtype='datetime64[M]')

In [11]:
print('Theoretical num. months:', 12*4)
print('Number of months:', len(tmp))

Theoretical num. months: 48
Number of months: 48


In [12]:
print(base['Category'].unique())

['Office Supplies' 'Furniture' 'Technology']


## Working with the Pandas DatetimeIndex

In [13]:
# Way easier to work with DatetimeIndex
base.set_index('Order Date', inplace= True)

In [14]:
base

,Category,Sales
Order Date,,
2011-01-04,Office Supplies,16.45
2011-01-05,Office Supplies,288.06
2011-01-06,Office Supplies,19.54
2011-01-07,Furniture,"2,573.82"
2011-01-07,Office Supplies,685.34
...,...,...
2014-12-30,Office Supplies,282.44
2014-12-30,Technology,302.38
2014-12-31,Furniture,323.14


In [22]:
display(base.loc['2011'].tail())
display(base[base['Category'] == 'Technology'].loc['2011'])
display(base[base['Category'] == 'Technology'].loc['2011':'2012-04'])
display(base[(base['Category'] == 'Technology') | (base['Category'] == 'Furniture')].loc['2014'])

,Category,Sales
Order Date,,
2011-12-30,Office Supplies,774.61
2011-12-30,Technology,874.28
2011-12-31,Furniture,"2,584.53"
2011-12-31,Office Supplies,308.73
2011-12-31,Technology,"2,360.01"


,Category,Sales
Order Date,,
2011-01-07,Technology,"1,147.94"
2011-01-10,Technology,31.20
2011-01-14,Technology,646.74
2011-01-16,Technology,149.95
2011-01-17,Technology,124.20
...,...,...
2011-12-24,Technology,"1,054.88"
2011-12-27,Technology,498.00
2011-12-29,Technology,"1,423.67"


,Category,Sales
Order Date,,
2011-01-07,Technology,"1,147.94"
2011-01-10,Technology,31.20
2011-01-14,Technology,646.74
2011-01-16,Technology,149.95
2011-01-17,Technology,124.20
...,...,...
2012-04-21,Technology,469.95
2012-04-22,Technology,88.78
2012-04-26,Technology,311.98


,Category,Sales
Order Date,,
2014-01-01,Technology,302.38
2014-01-02,Furniture,975.49
2014-01-03,Furniture,913.43
2014-01-03,Technology,712.29
2014-01-08,Technology,"3,059.98"
...,...,...
2014-12-29,Technology,14.85
2014-12-30,Furniture,"2,330.72"
2014-12-30,Technology,302.38


In [16]:
# Day of week= Monday=0, Sunday= 6
base['DayofWeek']= base.index.dayofweek 
display(base)
del(base['DayofWeek'])

,Category,Sales,DayofWeek
Order Date,,,
2011-01-04,Office Supplies,16.45,1
2011-01-05,Office Supplies,288.06,2
2011-01-06,Office Supplies,19.54,3
2011-01-07,Furniture,"2,573.82",4
2011-01-07,Office Supplies,685.34,4
...,...,...,...
2014-12-30,Office Supplies,282.44,1
2014-12-30,Technology,302.38,1
2014-12-31,Furniture,323.14,2


## Standardizing the DatetimeIndex
Some time-series applications require that data contain all periods and have a frequency assigned (freq= None)   
We need to ensure there are:
* No duplicate index values: **Pivot data**
* No missing index values

In [23]:
display(base.index)

DatetimeIndex(['2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
               '2011-01-07', '2011-01-07', '2011-01-08', '2011-01-08',
               '2011-01-10', '2011-01-10',
               ...
               '2014-12-28', '2014-12-29', '2014-12-29', '2014-12-29',
               '2014-12-30', '2014-12-30', '2014-12-30', '2014-12-31',
               '2014-12-31', '2014-12-31'],
              dtype='datetime64[ns]', name='Order Date', length=2864, freq=None)

In [24]:
base.reset_index(inplace= True)
sales= base.pivot(index= 'Order Date', columns= 'Category', values= 'Sales').fillna(0)
sales

Category,Furniture,Office Supplies,Technology
Order Date,,,
2011-01-04,0.00,16.45,0.00
2011-01-05,0.00,288.06,0.00
2011-01-06,0.00,19.54,0.00
2011-01-07,"2,573.82",685.34,"1,147.94"
2011-01-08,76.73,10.43,0.00
...,...,...,...
2014-12-27,0.00,814.59,0.00
2014-12-28,0.00,13.25,164.39
2014-12-29,551.26,"1,091.24",14.85


In [25]:
print(sales.index)
print('Unique values:', len(sales.index.unique()))

DatetimeIndex(['2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
               '2011-01-08', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-14', '2011-01-15',
               ...
               '2014-12-22', '2014-12-23', '2014-12-24', '2014-12-25',
               '2014-12-26', '2014-12-27', '2014-12-28', '2014-12-29',
               '2014-12-30', '2014-12-31'],
              dtype='datetime64[ns]', name='Order Date', length=1238, freq=None)
Unique values: 1238


## Generating a complete Index and Setting Frequency
Since we're using daily data, we would like to set a daily frequency

In [26]:
print(len(sales.index.unique()))
print(sales.index.min())
print(sales.index.max())
date_range= sales.index.max() - sales.index.min()
date_range

1238
2011-01-04 00:00:00
2014-12-31 00:00:00


Timedelta('1457 days 00:00:00')

In [27]:
new_index= pd.date_range(sales.index.min(), sales.index.max())
new_index

DatetimeIndex(['2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
               '2011-01-08', '2011-01-09', '2011-01-10', '2011-01-11',
               '2011-01-12', '2011-01-13',
               ...
               '2014-12-22', '2014-12-23', '2014-12-24', '2014-12-25',
               '2014-12-26', '2014-12-27', '2014-12-28', '2014-12-29',
               '2014-12-30', '2014-12-31'],
              dtype='datetime64[ns]', length=1458, freq='D')

In [28]:
new_sales= sales.reindex(new_index, fill_value= 0)
new_sales

Category,Furniture,Office Supplies,Technology
2011-01-04,0.00,16.45,0.00
2011-01-05,0.00,288.06,0.00
2011-01-06,0.00,19.54,0.00
2011-01-07,"2,573.82",685.34,"1,147.94"
2011-01-08,76.73,10.43,0.00
...,...,...,...
2014-12-27,0.00,814.59,0.00
2014-12-28,0.00,13.25,164.39
2014-12-29,551.26,"1,091.24",14.85
2014-12-30,"2,330.72",282.44,302.38


We can observe we have a daily frequency

In [29]:
new_sales.index

DatetimeIndex(['2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
               '2011-01-08', '2011-01-09', '2011-01-10', '2011-01-11',
               '2011-01-12', '2011-01-13',
               ...
               '2014-12-22', '2014-12-23', '2014-12-24', '2014-12-25',
               '2014-12-26', '2014-12-27', '2014-12-28', '2014-12-29',
               '2014-12-30', '2014-12-31'],
              dtype='datetime64[ns]', length=1458, freq='D')

In [33]:
new_sales[(new_sales['Furniture'] == 0) & (new_sales['Technology'] == 0) & (new_sales['Office Supplies'] == 0)]

Category,Furniture,Office Supplies,Technology
2011-01-09,0.00,0.00,0.00
2011-01-13,0.00,0.00,0.00
2011-01-18,0.00,0.00,0.00
2011-01-23,0.00,0.00,0.00
2011-01-25,0.00,0.00,0.00
...,...,...,...
2014-09-28,0.00,0.00,0.00
2014-10-12,0.00,0.00,0.00
2014-10-19,0.00,0.00,0.00
2014-10-26,0.00,0.00,0.00


In [131]:
# new_sales.to_csv('processed_df/processed.df.csv')